In [1]:
import pandas_datareader.data as web
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
#plt.style.use('ggplot')
%matplotlib inline

In [2]:
etf_df = pd.read_csv("resources/etf_df.csv")

### Period Datasets
1M, 3M, 6M, 1Y, 5Y, 10Y, and MAX

In [3]:
# last date configured
last_date = list(etf_df.Date)[-1]
last_split = last_date.split('-')
d_list = [int(i) for i in last_split]
date = dt.date(d_list[0], d_list[1], d_list[2])

In [4]:
# calculate the dates from the last data point for 1m, 3m, 6m, 1y, 5y, and 10y...
# ...then convert to string format for filtering DataFrame
one_mo = (date - dt.timedelta(days=30.4167)).strftime("%Y-%m-%d")
three_mo = (date - dt.timedelta(days=91.2501)).strftime("%Y-%m-%d")
six_mo = (date - dt.timedelta(days=182.5)).strftime("%Y-%m-%d")
one_yr = (date - dt.timedelta(days=365)).strftime("%Y-%m-%d")
five_yr = (date - dt.timedelta(days=1825)).strftime("%Y-%m-%d")
ten_yr = (date - dt.timedelta(days=3650)).strftime("%Y-%m-%d")

In [5]:
etf_df.loc[(etf_df.symbol == 'QQQ')&(etf_df.Date >= ten_yr)&(etf_df.Date <= last_date)]

,Date,Open,High,Low,Close,Adj Close,Volume,symbol
3033,2011-03-29,56.400002,57.080002,56.230000,57.080002,51.599754,47994800,QQQ
3034,2011-03-30,57.360001,57.430000,57.080002,57.349998,51.843811,44935900,QQQ
3035,2011-03-31,57.250000,57.480000,57.150002,57.430000,51.916138,50004300,QQQ
3036,2011-04-01,57.720001,57.900002,57.290001,57.459999,51.943260,68031800,QQQ
3037,2011-04-04,57.619999,57.680000,57.020000,57.270000,51.771496,48659100,QQQ
...,...,...,...,...,...,...,...,...
5544,2021-03-22,315.209991,320.739990,315.209991,318.609985,318.609985,56194800,QQQ
5545,2021-03-23,319.730011,321.089996,316.380005,317.220001,317.220001,53455400,QQQ
5546,2021-03-24,318.630005,318.670013,311.720001,311.869995,311.869995,64803700,QQQ
5547,2021-03-25,309.970001,312.890015,307.390015,311.329987,311.329987,78729000,QQQ


### 1. Relative Returns

In [6]:
# three lists created to cover possible portfolio combinations to display
qqq_symbols = ['QQQ', 'SPLG', 'DIA', 'VEU']
sp_symbols = ['QQQ', 'SPLG', 'IVV', 'VOO', 'SPY', 'DIA']
glob_symbols = ['QQQ', 'SPLG', 'ACWI', 'VEU', 'VSS']

#list of lists...
portfolios = [qqq_symbols, sp_symbols, glob_symbols]

# list of years we need a portfolio display for
dates_list = [one_mo, three_mo, six_mo, one_yr, five_yr, ten_yr]
dates_str = ['1M', '3M', '6M', '1Y', '5Y', '10Y']

In [7]:
# create function that exports a DataFrame for the list of symbols' relative returns
def rel_returns(symbols, start):
    df = pd.DataFrame(columns = symbols)
    for symbol in symbols:
        df_filter = etf_df.loc[(etf_df.symbol == symbol)&(etf_df.Date >= start)&(etf_df.Date <= last_date)]
        df[symbol] = list(df_filter.Close)
    df['Date'] = list(df_filter.Date)
    df.set_index("Date", inplace = True)
    returns = (df/df.iloc[0])
    return returns

def close_compare(symbols, start):
    df = pd.DataFrame(columns = symbols)
    for symbol in symbols:
        df_filter = etf_df.loc[(etf_df.symbol == symbol)&(etf_df.Date >= start)&(etf_df.Date <= last_date)]
        df[symbol] = list(df_filter.Close)
    df['Date'] = list(df_filter.Date)
    df.set_index("Date", inplace = True)
    return df

In [8]:
# create function that exports two charts for every period specified in dates_list...
# ...one for Close price and one for relative returns
def returns_charts(symbols):
    for x in range(len(dates_list)):
        new_df = rel_returns(symbols, dates_list[x])
        new_df.plot(figsize=(10,8))
        plt.title(f'Relative Returns: {dates_str[x]}, {last_date}', fontsize=24)
        plt.xlabel('Date', fontsize=22)
        plt.legend(fontsize=18)
        plt.grid()
        plt.tight_layout()
        plt.savefig(f"../images/returns/returns_{symbols[-1]}_{dates_str[x]}.jpg", dpi=75)
        plt.clf()
        
def close_charts(symbols):
    for x in range(len(dates_list)):
        new_df = close_compare(symbols, dates_list[x])
        new_df.plot(figsize=(10,8))
        plt.title(f'Close: {dates_str[x]}, {last_date}', fontsize=24)
        plt.xlabel('Date', fontsize=22)
        plt.legend(fontsize=18)
        plt.grid()
        plt.tight_layout()
        plt.savefig(f"../images/close/close_{symbols[-1]}_{dates_str[x]}.jpg", dpi=75)
        plt.clf()

In [9]:
returns_charts(qqq_symbols)
returns_charts(sp_symbols)
returns_charts(glob_symbols)

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

In [10]:
close_charts(qqq_symbols)
close_charts(sp_symbols)
close_charts(glob_symbols)

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

### 2. Volatility - Histograms

In [11]:
df = rel_returns(qqq_symbols, ten_yr)

In [12]:
df

,QQQ,SPLG,DIA,VEU
Date,,,,
2011-03-29,1.000000,1.000000,1.000000,1.000000
2011-03-30,1.004730,1.007435,1.005714,1.011478
2011-03-31,1.006132,1.006788,1.004163,1.010863
2011-04-01,1.006657,1.009375,1.007754,1.022341
2011-04-04,1.003329,1.011637,1.010203,1.026030
...,...,...,...,...
2021-03-22,5.581815,2.983676,2.671129,1.254765
2021-03-23,5.557463,2.961047,2.645172,1.237139
2021-03-24,5.463735,2.945531,2.645335,1.223201


In [13]:
# create histograms for returns, qqq_symbols list
for x in range(len(dates_list)):
    df = rel_returns(qqq_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.DIA.hist(bins=50, label='DIA', alpha=0.5)
    df.VEU.hist(bins=50, label='VEU', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_qqq_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [14]:
# create histograms kernel for clarity, qqq_symbols list
for x in range(len(dates_list)):
    df = rel_returns(qqq_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.DIA.plot(kind='kde', label='DIA')
    df.VEU.plot(kind='kde', label='VEU')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_qqq_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [15]:
# repeat operations for sp_symbols, glob_symbols
for x in range(len(dates_list)):
    df = rel_returns(sp_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.IVV.hist(bins=50, label='IVV', alpha=0.5)
    df.VOO.hist(bins=50, label='VOO', alpha=0.5)
    df.SPY.hist(bins=50, label='SPY', alpha=0.5)
    df.DIA.hist(bins=50, label='DIA', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_sp_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()
for x in range(len(dates_list)):
    df = rel_returns(sp_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.IVV.plot(kind='kde', label='IVV')
    df.VOO.plot(kind='kde', label='VOO')
    df.SPY.plot(kind='kde', label='SPY')
    df.DIA.plot(kind='kde', label='DIA')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_sp_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

In [16]:
for x in range(len(dates_list)):
    df = rel_returns(glob_symbols, dates_list[x])
    df.QQQ.hist(bins=50, figsize=(10,8), label='QQQ', alpha=0.5)
    df.SPLG.hist(bins=50, label='SPLG', alpha=0.5)
    df.ACWI.hist(bins=50, label='ACWI', alpha=0.5)
    df.VEU.hist(bins=50, label='VEU', alpha=0.5)
    df.VSS.hist(bins=50, label='VSS', alpha=0.5)
    plt.title(f'Volatility: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(f"../images/vol_hist/histogram_glob_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()
for x in range(len(dates_list)):
    df = rel_returns(glob_symbols, dates_list[x])
    df.QQQ.plot(kind='kde', label='QQQ', figsize=(10,8))
    df.SPLG.plot(kind='kde', label='SPLG')
    df.ACWI.plot(kind='kde', label='ACWI')
    df.VEU.plot(kind='kde', label='VEU')
    df.VSS.plot(kind='kde', label='VSS')
    plt.title(f'Volatility Normalized: {dates_str[x]}, {last_date}', fontsize=24)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"../images/vol_hist/ked_hist_glob_symbols_{dates_str[x]}.jpg", dpi=75)
    plt.clf()

<Figure size 720x576 with 0 Axes>

### Correlation

In [17]:
from pandas.plotting import scatter_matrix

In [18]:
def scatter_plots(portfolio_list):
    for portfolio in portfolio_list:
        for x in range(len(dates_list)):
            df = close_compare(portfolio, dates_list[x])
            scatter_matrix(df, figsize=(10,10), color='dodgerblue', hist_kwds={'color':'lightblue', 'alpha':0.50, 'bins':50})
            plt.tight_layout()
            plt.savefig(f"../images/correlation/scatter_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=150)
            plt.clf()

In [19]:
scatter_plots(portfolios)

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

### Monte Carlo Simulation
Currently discontinued

In [20]:
# create function that randomly selects a 60 day period 1000 times, plots the simulation and...
# an easier to read quantile version
#def monte_carlo(portfolio):
    #for x in range(len(dates_list)):
        #stocks = close_compare(portfolio, dates_list[x])
        #prf_returns = (stocks.pct_change() +1)[1:]
        #simulated = pd.DataFrame([((prf_returns.iloc[random.choices(range(len(prf_returns)), k=60)])\
                        #.mean(axis=1)).cumprod().values for x in range(1000)]).T
        #simulated.plot(legend=False, linewidth=1, alpha=0.1, color='blue', figsize=(12,8))
        #plt.tight_layout()
        #plt.savefig(f"../images/monte_carlo/simulated_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=150)
        #plt.clf()
        #simulated.quantile([0.05, 0.50, 0.95], axis=1).T.plot(figsize=(12,8))
        #plt.legend(fontsize=18)
        #plt.tight_layout()
        #plt.savefig(f"../images/monte_carlo/quantile_{portfolio[-1]}_{dates_str[x]}.jpg", dpi=75)
        #plt.clf()

In [21]:
#monte_carlo(qqq_symbols)

In [22]:
#monte_carlo(sp_symbols)

In [23]:
#monte_carlo(glob_symbols)